<p><img src="images/1line.png" width="100%"  /></p>
<ul>
<li>The example below utilizes Businesses formation information from a Colorado site for Colorado Business entities.</li>
<li>Data Site: <a href="https://data.colorado.gov/resource/4ykn-tg5h.json">https://data.colorado.gov/resource/4ykn-tg5h.json</a>&nbsp;</li>
<li>Data JSON Format (one entity):</li>
</ul>
<pre>{	"entityid": "18861217679",
	"entityname": "DENVER UNION CORPROATION, Dissolved January 17, 1983",
	"principaladdress1": "1512 LARIMER STREET #760",
	"principalcity": "Denver",
	"principalstate": "CO",
	"principalzipcode": "80202",
	"entitystatus": "Voluntarily Dissolved",
	"jurisdictonofformation": "CO",
	"entitytype": "Corporation",
	"agentfirstname": "JOHN",
	"agentmiddlename": "F.",
	"agentlastname": "O'DEA",
	"agentprincipaladdress1": "1512 LARIMER STREET #760",
	"agentprincipalcity": "DENVER",
	"agentprincipalstate": "CO",
	"agentprincipalzipcode": "80202",
	"agentprincipalcountry": "US",
	"entityformdate": "1886-03-29T00:00:00.000"  }</pre>
<ul>
<li>Records are downloaded from the website for the 500 most recent business entities opened in the zip code "80211".</li>
<li>The JSON data is written to a SQLite database so the data can be stored locally.</li>
<li>It illustrates a common requirement for database programs the need to write data to multiple tables - which typically requires you to use the primary key field from one insert as part of the data for the next insert.
<ul>
<li>We get the primary key in the example below using the<span>&nbsp;</span><code>lastrowid</code><span>&nbsp;</span><span>property of the Cursor object.</span></li>
</ul>
</li>
<li><span>It also illustrates the </span><code>executescript()</code><span>&nbsp;method which allows you to execute a whole series of SQL statements&nbsp; (saved in one long string) in one step.</span></li>
<li>The program "Processes:" the JSON data by
<ul>
<li>Looping through the JSON array containing 500 Business entities</li>
<li>Extracting multiple pieces of data for the current business entity (from variable biz which is a Python dictionary)</li>
<li>Converting the ISO formatted timestamp to a Python date</li>
<li>Inserting the principleaddress into the Address table and getting the addressid</li>
<li>Inserting the agentprincipleaddress into the Address table and getting the addressid - in a try/except because every business does not have an agentprinciple.</li>
<li>Inserting the rest of the CoBiz data into the CoBiz table (including the two address ids).</li>
</ul>
</li>
<li>We get save the changes to the database in the example below using the<span>&nbsp;</span><code>commit</code><span>&nbsp;</span><span>method of the Connection object AND we only save every 100 records so as not to slow down the program too much. <br />REMEMBER IF YOU DON"T COMMIT THE DATA IS NOT SAVED!!!</span></li>
</ul>

In [1]:
# -*- coding: utf-8 -*-
"""
Colorado Businesses downloader python program
Created on Fri November 29 12:07:50 2021

@author: Dawn Gregg
"""
import json
import requests
import sqlite3
import datetime

# connect to the output database and name it index.sqlite
conn = sqlite3.connect('COBiz.sqlite')
# forces database to retun strings for TEXT attributes
conn.text_factory = str
# get the cursor for the connection
cur = conn.cursor()

# Drop the Tables each time the code is run so you capture changes made to the way data is processed
cur.executescript('''DROP TABLE IF EXISTS CoBiz;
                     DROP TABLE IF EXISTS Address''')

# the Create Table query should include all of the columns you need to process your data
cur.executescript('''CREATE TABLE IF NOT EXISTS CoBiz (entityid TEXT PRIMARY KEY, 
    entityname Text, principaladdress INTEGER, agentprincipaladdress INTEGER, 
    entitystatus TEXT, entitytype TEXT, entityformdate TEXT, year INTEGER);
    CREATE TABLE IF NOT EXISTS Address 
    (addressid INTEGER PRIMARY KEY AUTOINCREMENT, address1 Text, city TEXT, 
    state TEXT, zipcode TEXT, country TEXT)''')

# Initial settings that control the data download
cobizURL =  "https://data.colorado.gov/resource/4ykn-tg5h.json"
paramD = dict()
paramD["principalzipcode"] = 80211                            
paramD["$order"] = "entityformdate DESC"
paramD["$limit"] = 500

# get a document based on base URL plus parameters
document = requests.get(cobizURL, paramD)

# get the JSON text from the URL into a dictionary using the requests library       
if document.status_code == 200 :
    print(document.request.url)
    js = document.json()
else:
    print("Error code=",document.status_code, document.requests.url)
    js = json.loads("{}")

# Output first Record
print("\nFirst Business Reading")
print(js[0])

count = 0
# Loop through entire data set
print("\nAll Entities Downloaded for", paramD["principalzipcode"])
for biz in js:    
    isodate = biz["entityformdate"]
    dt = datetime.datetime.fromisoformat(isodate)
    timestamp = datetime.datetime.timestamp(dt)
          
    # principal address         
    address = biz["principaladdress1"]
    city    = biz["principalcity"]
    zip     = biz["principalzipcode"]
    state   = biz["principalstate"]
    country = biz["principalcountry"]

    # the insert query should include all of the columns you need to add your data
    cur.execute('''INSERT INTO Address (address1, city, state, zipcode, country) 
     VALUES (?,?,?,?,?)''', (address, city, state, zip, country))
    prinaddress_id = cur.lastrowid
         
    try : #Error handling
        # agent principal address         
        address = biz["agentprincipaladdress1"]
        city    = biz["agentprincipalcity"]
        zip     = biz["agentprincipalzipcode"]
        state   = biz["agentprincipalstate"]
        country = biz["agentprincipalcountry"]
         
        # the insert query should include all of the columns you need to add your data
        cur.execute('''INSERT INTO Address (address1, city, state, zipcode, country) 
         VALUES (?,?,?,?,?)''', (address, city, state, zip, country))
        agentaddress_id = cur.lastrowid
    except:
        print("\nError: No Agent Principle for", biz["entityname"])
        # the insert query creates an empty address
        cur.execute('''INSERT OR IGNORE INTO Address (addressid) VALUES (?)''', (0,))
        agentaddress_id = 0
     
    # the insert query should include all of the columns you need to add your data
    cur.execute('''INSERT INTO CoBiz (entityid, entityname, principaladdress, agentprincipaladdress, 
     entitystatus, entitytype, entityformdate, year)  VALUES (?,?,?,?,?,?,?,?)''',
     (biz["entityid"], biz["entityname"], prinaddress_id, agentaddress_id,
     biz["entitystatus"], biz["entitytype"], timestamp, dt.year))

    #increment count
    count = count+1    
         
    # commit the insert statemnt(s) for every 100 records so do not slow program down       
    if count%100==0: 
        print("\nDate Business Started:", isodate)
        print("Business Name:", biz["entityname"])
        conn.commit()

# Remember to commit after the end of the loop to make sure ALL data is saved!!    
print(count,"Colorado Businesses Processed")
conn.commit()

https://data.colorado.gov/resource/4ykn-tg5h.json?principalzipcode=80211&%24order=entityformdate+DESC&%24limit=500

First Business Reading
{'entityid': '20231647489', 'entityname': 'Misfit Studios LLC', 'principaladdress1': '2600 W 29th Ave Unit A', 'principalcity': 'Denver', 'principalstate': 'CO', 'principalzipcode': '80211', 'principalcountry': 'US', 'entitystatus': 'Good Standing', 'jurisdictonofformation': 'CO', 'entitytype': 'Limited Liability Company', 'agentorganizationname': 'Misfit Studios LLC', 'agentprincipaladdress1': '2600 W 29th Ave Unit A', 'agentprincipalcity': 'Denver', 'agentprincipalstate': 'CO', 'agentprincipalzipcode': '80211', 'agentprincipalcountry': 'US', 'entityformdate': '2023-06-22T00:00:00.000'}

All Entities Downloaded for 80211

Date Business Started: 2023-05-30T00:00:00.000
Business Name: AFFORDABLE CARDS AND COLLECTABLES LLC

Date Business Started: 2023-05-07T00:00:00.000
Business Name: Jansen Family Revocable Trust LLC

Date Business Started: 2023-04-1